In diesem Notebook werden zwei Streaming Klassifizierungen mit 10.000 Instanzen durchgeführt:

1. Klassifizierung der agr_g_t Daten mit verspäteten Eintreffen der Label
2. Klassifizierung der agr_g Daten

In [1]:
import numpy as np
import pandas as pd

from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.bayes import NaiveBayes
from skmultiflow.meta import AccuracyWeightedEnsembleClassifier
from skmultiflow.lazy import KNNADWINClassifier

from skmultiflow.data import TemporalDataStream
from skmultiflow.data.file_stream import FileStream
from skmultiflow.data.data_stream import DataStream

from skmultiflow.evaluation import EvaluatePrequentialDelayed
from skmultiflow.evaluation import EvaluatePrequential

In [2]:
import warnings
warnings.filterwarnings('ignore')

#set columns
attribute = ['salary', 'comission', 'age', 'elevel',
                 'car', 'zipcode', 'hvalue', 'hyears', 'loan']
target = "class"
timestamp = "datetime"

#read csv
t_agr_g1 = pd.read_csv("data/t_agr_g1.csv")

# Convert time to datetime
t_agr_g1[timestamp] = pd.to_datetime(t_agr_g1[timestamp])

#get X, y and time
X = t_agr_g1[attribute].values
y = t_agr_g1[target].values
time = t_agr_g1[timestamp].values

#set a delay of 1 day
delay_time = np.timedelta64(1, "D")

#create stream
t_agr_g1_stream = TemporalDataStream(X, y, time, sample_delay=delay_time, ordered=False)
nominal=['elevel', 'car', 'zipcode']

#set classifier
ht = HoeffdingTreeClassifier(leaf_prediction='nb', nominal_attributes=nominal)
hat = HoeffdingAdaptiveTreeClassifier(nominal_attributes=nominal)
efdt = ExtremelyFastDecisionTreeClassifier(nominal_attributes=nominal)
arf = AdaptiveRandomForestClassifier(nominal_attributes=nominal)
snb = NaiveBayes(nominal_attributes=nominal)
awe = AccuracyWeightedEnsembleClassifier(n_estimators=15, base_estimator=NaiveBayes(nominal_attributes=nominal))
knn_adwin = KNNADWINClassifier(n_neighbors=10)

# Set evaluator
evaluator = EvaluatePrequentialDelayed(
                                max_samples=X.shape[0],
                                n_wait=10,
                                pretrain_size=100,
                                output_file='results_t_agr_g1.csv',
                                metrics=['accuracy', 'kappa', 
                                         'precision', 'recall', 
                                         'f1', 'running_time', 
                                         'model_size'])

#evaluate
evaluator.evaluate(stream=t_agr_g1_stream, 
                   model=[ht, hat, efdt, arf, 
                          snb, awe, knn_adwin], 
                   model_names=['Hoeffding Tree', 'Hoeffding Adaptive Tree', 
                                'Extremely Fast Decision Tree', 
                                'Adaptive Random Forest', 
                                'Naive Bayes', 'Accuracy Weighted Ensembler', 'KNN ADWIN'])

Prequential Evaluation Delayed
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...
 ###################- [95%] [504.73s]Processed samples: 10000
Mean performance:
Hoeffding Tree - Accuracy     : 0.4694
Hoeffding Tree - Kappa        : -0.0354
Hoeffding Tree - Precision: 0.5932
Hoeffding Tree - Recall: 0.4304
Hoeffding Tree - F1 score: 0.4989
Hoeffding Tree - Training time (s)  : 1.00
Hoeffding Tree - Testing time  (s)  : 1.31
Hoeffding Tree - Total time    (s)  : 2.31
Hoeffding Tree - Size (kB)          : 63.1826
Hoeffding Adaptive Tree - Accuracy     : 0.4631
Hoeffding Adaptive Tree - Kappa        : -0.0182
Hoeffding Adaptive Tree - Precision: 0.6010
Hoeffding Adaptive Tree - Recall: 0.3722
Hoeffding Adaptive Tree - F1 score: 0.4597
Hoeffding Adaptive Tree - Training time (s)  : 3.63
Hoeffding Adaptive Tree - Testing time  (s)  : 1.08
Hoeffding Adaptive Tree - Total time    (s)  : 4.71
Hoeffding Adaptive Tree - Size (kB)          : 60.1299
Extremely Fast Decision Tree 

[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nb', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False,
                         nominal_attributes=['elevel', 'car', 'zipcode'],
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_preprune=False,
                                 nominal_attributes=['elevel', 'car', 'zipcode'],
                                 random_state=None,

In [3]:
#drop timestamp
t_agr_g1 = pd.read_csv('data/t_agr_g1.csv')
agr_g1 = t_agr_g1.drop(['datetime'], axis=1)

#create stream
agr_g1_stream = DataStream(agr_g1)

#set classifier
ht = HoeffdingTreeClassifier(leaf_prediction='nb', nominal_attributes=nominal)
hat = HoeffdingAdaptiveTreeClassifier(nominal_attributes=nominal)
efdt = ExtremelyFastDecisionTreeClassifier(nominal_attributes=nominal)
arf = AdaptiveRandomForestClassifier(nominal_attributes=nominal)
snb = NaiveBayes(nominal_attributes=nominal)
awe = AccuracyWeightedEnsembleClassifier(n_estimators=15, base_estimator=NaiveBayes(nominal_attributes=nominal))
knn_adwin = KNNADWINClassifier(n_neighbors=10)

#set evaluator
evaluator = EvaluatePrequential(
                                max_samples=10000,
                                n_wait=10,
                                pretrain_size=100,
                                metrics=['accuracy', 'kappa', 
                                         'precision', 'recall', 
                                         'f1', 'running_time', 
                                         'model_size'])

#evaluate
evaluator.evaluate(stream=agr_g1_stream, 
                   model=[ht, hat, efdt, arf, 
                          snb, awe, knn_adwin], 
                   model_names=['Hoeffding Tree', 'Hoeffding Adaptive Tree', 
                                'Extremely Fast Decision Tree', 
                                'Adaptive Random Forest', 
                                'Naive Bayes', 'Accuracy Weighted Ensembler', 'KNN ADWIN' ])

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...
 #################### [100%] [249.61s]
Processed samples: 10000
Mean performance:
Hoeffding Tree - Accuracy     : 0.7297
Hoeffding Tree - Kappa        : 0.4076
Hoeffding Tree - Precision: 0.7474
Hoeffding Tree - Recall: 0.8447
Hoeffding Tree - F1 score: 0.7931
Hoeffding Tree - Training time (s)  : 0.88
Hoeffding Tree - Testing time  (s)  : 1.23
Hoeffding Tree - Total time    (s)  : 2.11
Hoeffding Tree - Size (kB)          : 71.7100
Hoeffding Adaptive Tree - Accuracy     : 0.7372
Hoeffding Adaptive Tree - Kappa        : 0.4272
Hoeffding Adaptive Tree - Precision: 0.7570
Hoeffding Adaptive Tree - Recall: 0.8415
Hoeffding Adaptive Tree - F1 score: 0.7970
Hoeffding Adaptive Tree - Training time (s)  : 4.80
Hoeffding Adaptive Tree - Testing time  (s)  : 1.04
Hoeffding Adaptive Tree - Total time    (s)  : 5.84
Hoeffding Adaptive Tree - Size (kB)          : 61.4385
Extremely Fast Decision Tree - Accura

[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nb', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False,
                         nominal_attributes=['elevel', 'car', 'zipcode'],
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_preprune=False,
                                 nominal_attributes=['elevel', 'car', 'zipcode'],
                                 random_state=None,